**Face Recognition using Transfer Learning from FaceNet**

In [2]:
!pip install keras-facenet

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
from keras_facenet import FaceNet
embedder = FaceNet()

Using TensorFlow backend.














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
import matplotlib.image as mpimg 
import cv2
import numpy as np
import pandas as pd

In [0]:
'''

'''

In [0]:
def load_train_data(n):
  lst = []
  for i in range(1,n+1):
    filename = str(i)+'.jpg'
    x = mpimg.imread('/content/drive/My Drive/face reco/train/'+filename)
    x = cv2.resize(x,(160,160))
    lst.append(x)
  df = np.array(lst)
  y_train = pd.read_csv('/content/drive/My Drive/face reco/train/y_train.csv',header=None)
  y_train = y_train.values
  return df,y_train

In [0]:
def load_test_data(n):
  lst = []
  for i in range(1,n+1):
    filename = str(i)+'.jpg'
    x = mpimg.imread('/content/drive/My Drive/face reco/Test/'+filename)
    x = cv2.resize(x,(160,160))
    lst.append(x)
  df = np.array(lst)
  y_test = pd.read_csv('/content/drive/My Drive/face reco/Test/y_test.csv',header=None)
  y_test = y_test.values
  return df,y_test

In [17]:
'''
Loading the data set 
'''
X_train_orig,y_train = load_train_data(24)
X_test_orig,y_test = load_test_data(7)
print('X_Train data shape=',X_train_orig.shape)
print('X_Test data shape=',X_test_orig.shape)
print('y_Train data shape=',y_train.shape)
print('y_Test data shape=',y_test.shape)

X_Train data shape= (24, 160, 160, 3)
X_Test data shape= (7, 160, 160, 3)
y_Train data shape= (24, 1)
y_Test data shape= (7, 1)


In [18]:
X_train = embedder.embeddings(X_train_orig)
X_test = embedder.embeddings(X_test_orig)
print('Train embed shape=',X_train.shape)
print('Test embed shape=',X_test.shape)

Train embed shape= (24, 512)
Test embed shape= (7, 512)


In [0]:
from sklearn.svm import SVC

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
model = LogisticRegression()

In [31]:
model.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [65]:
#Predicted faces
model.predict(X_test)

array([1, 2, 2, 3, 3, 4, 6])

In [68]:
#True faces
y_test

array([[1],
       [2],
       [2],
       [3],
       [3],
       [4],
       [6]])

In [72]:
model.score(X_test,y_test)

1.0

In [0]:
'''
To recognise face of a given input file with path and filename.
Returns one of the six classes if face is from the training data faces,
else 'Face not found !' message is returned .
'''

def predict(path, filename):
  x = mpimg.imread(path+'/'+filename)
  x = cv2.resize(x,(160,160))
  x = x.reshape(1,160,160,3)
  x = embedder.embeddings(x)
  if(max(model.predict_proba(x)[0])<0.25):         #0.25 is the threshold
    print("Face not found !")
    return
  else:
    return model.predict(x)